In [1]:
# import os
# import pandas as pd
# from docx import Document
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report

# # Initialize an empty list to hold data
# data = []

# # Define the base directory
# base_dir = "Тренировочные данные (train)"

# # Function to read .docx file
# def read_docx(file_path):
#     doc = Document(file_path)
#     full_text = []
#     for para in doc.paragraphs:
#         full_text.append(para.text)
#     return '\n'.join(full_text)

# # Walk through the base directory and its subfolders
# for root, dirs, files in os.walk(base_dir):
#     # Get the folder name, which will be used as the class label
#     folder_name = os.path.basename(root)
    
#     for file in files:
#         if file.endswith(".docx"):
#             file_path = os.path.join(root, file)
#             # Read the text from the .docx file
#             text = read_docx(file_path)
#             # Append the text and the class (folder name) to the data list
#             data.append([file, text, folder_name])

# # Create a DataFrame
# df = pd.DataFrame(data, columns=['filename', "text", "class"])

# # Save the DataFrame to an Excel file for reference
# df.to_excel('texts.xlsx', index=False)

In [2]:
# ! pip install transformers torch


In [8]:
import os

from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

import torch



In [9]:
# # Install necessary packages and download model
# !pip install mistral-inference
# !wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-Instruct-v0.3.tar

In [10]:
# # Verify the downloaded file exists
# !ls /home/user1/atom_hackathon/mistral-7B-Instruct-v0.3.tar

In [11]:
# # Extract the tar file to the specified directory
# !DIR=/home/user1/atom_hackathon/mistral_7b_instruct_v3 && mkdir -p $DIR && tar -xf /home/user1/atom_hackathon/mistral-7B-Instruct-v0.3.tar -C $DIR
# # Check the contents of the directory after extraction
# !ls $DIR

In [12]:
path_tokenizer = os.path.expanduser("~") + "/atom_hackathon/mistral_7b_instruct_v3/tokenizer.model.v3"
path_tokenizer

'/home/user1/atom_hackathon/mistral_7b_instruct_v3/tokenizer.model.v3'

In [13]:
path_model = os.path.expanduser("~") + "/atom_hackathon/mistral_7b_instruct_v3"
path_model

'/home/user1/atom_hackathon/mistral_7b_instruct_v3'

In [16]:
# Load tokenizer
mistral_tokenizer = MistralTokenizer.from_file(path_tokenizer)

# Define your regulation and use case texts
regulation_text = """
AVAS. Acoustics characteristics.
The sound emitted by the vehicle type submitted for approval shall be measured by the methods described in Annex 3 to this Regulation.
The specifications of this Regulation are applicable for the speed range of greater than 0 km/h up to and inclusive 20 km/h. Operation of an AVAS is permitted at vehicle speeds outside the specification range. AVAS may be operational independent of the operation of an internal combustion engine inside or outside of the specified operation range.
If the vehicle that is not equipped with an AVAS fulfils the overall levels as specified in Table 2 below with a margin of +3 dB(A), the specification for one-third octave bands and the frequency shift do not apply.an internal combustion engine inside or outside of the specified operation range. Frequency shift to signify acceleration and deceleration. The intention of frequency shift is to acoustically inform road users about the change in vehicle speed.
"""

usecase_text = """
 The driver is in the Vehicle, the Vehicle is in the R drive mode and reversing at any speed, an external soundtrack notifies surrounding road users about the movement of the Vehicle in reverse, so that an approaching Vehicle can be identified by ear. 
"""

# Construct the chat completion request with the specific prompt
prompt = f"I will send you regulation text and text of usecase. Usecase contains some requirements that should be preprocessed by you and you should say if these requirements are satisfied by the regulation text.\n\nRegulation text: {regulation_text}\n\nUsecase text: {usecase_text}"

completion_request = ChatCompletionRequest(messages=[UserMessage(content=prompt)])

# Encode message
tokens = mistral_tokenizer.encode_chat_completion(completion_request).tokens

# Load model
model = Transformer.from_folder(path_model)

# Generate results
out_tokens, _ = generate([tokens], model, max_tokens=256, temperature=0.0, eos_id=mistral_tokenizer.instruct_tokenizer.tokenizer.eos_id)

# Decode generated tokens
result = mistral_tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

# Output the result
print(result)


OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 31.73 GiB of which 17.44 MiB is free. Including non-PyTorch memory, this process has 31.71 GiB memory in use. Of the allocated memory 31.06 GiB is allocated by PyTorch, and 299.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [5]:
! pip install llama-cpp-python

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 21.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 KB 3.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3510578 sha256=9da7dd6a45f586e98fdbd26feeb690a6b9413a890ee8c3bac7f88bd1ee4ebae1
  Stored in directory: /home/user1/.cache/pip/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python


In [2]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="SanctumAI/Meta-Llama-3.1-8B-Instruct-GGUF",
	filename="meta-llama-3.1-8b-instruct.f16.gguf",
)

meta-llama-3.1-8b-instruct.f16.gguf:   0%|          | 0.00/16.1G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 29 key-value pairs and 292 tensors from /home/user1/.cache/huggingface/hub/models--SanctumAI--Meta-Llama-3.1-8B-Instruct-GGUF/snapshots/1cbd69377fb06c0a4719187fc31fdc7d4020ec3a/./meta-llama-3.1-8b-instruct.f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B


In [3]:


# Define the regulation and use case texts
regulation_text = """
Acoustics characteristics
The sound emitted by the vehicle type submitted for approval shall be measured by the methods described in Annex 3 to this Regulation.
The specifications of this Regulation are applicable for the speed range of greater than 0 km/h up to and inclusive 20 km/h. Operation of an AVAS is permitted at vehicle speeds outside the specification range. AVAS may be operational independent of the operation of an internal combustion engine inside or outside of the specified operation range.
If the vehicle that is not equipped with an AVAS fulfils the overall levels as specified in Table 2 below with a margin of +3 dB(A), the specification for one-third octave bands and the frequency shift do not apply.an internal combustion engine inside or outside of the specified operation range. 
Frequency shift to signify acceleration and deceleration
The intention of frequency shift is to acoustically inform road users about the change in vehicle speed.
"""

use_case_text = """
Preconditions - driving below approximately 32 km/h or while driving in reverse Main Scenario - Car emit sound out_27, because EV operate quietly, and this sound helps to alert pedestrians of your oncoming vehicle.
"""

# Define the types to use
types = [
    "The developed system is not subject to certification. Verification is not required.",
    "The case mentions certifiable objects, and the regulations are complied with. It is necessary to specify the regulations that this case affects in the development.",
    "The case mentions certifiable objects that are subject to certification restrictions, but they are not described in the case. The case needs to be supplemented with a description of the restrictions from the regulations.",
    "The case mentions certifiable objects, and the requirement for development contradicts (does not comply with) the certification regulations. Corrections need to be made."
]

# Create the prompt
prompt = f"""
The use case is like a hypothesis which has to be correct. To check if the use case is correct you have to read through the regulation text.

Regulation text: {regulation_text}
Use case text: {use_case_text}

Try to determine the type. And follow its instructions.
Types:
0. {types[0]}
1. {types[1]}
2. {types[2]}
3. {types[3]}
Answer briefly.
"""

# Generate the completion
response = llm.create_chat_completion(
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    max_tokens=1024  # Adjust this value as necessary
)


# Print the response
print(response["choices"][0]["message"]["content"])


llama_perf_context_print:        load time =   26726.98 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   414 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    86 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   99342.78 ms /   500 tokens


After analyzing the use case and regulation text, I conclude that:

**Type: 1. The case mentions certifiable objects, and the regulations are complied with.**

The use case mentions a specific scenario where a car emits a sound to alert pedestrians, which is related to the regulation text about acoustic characteristics of vehicles. The use case does not contradict any certification regulations, and the regulation text provides context for the sound emission specification.


In [ ]:
response

In [9]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Initialize the T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Your input text
input_text = """
Preconditions
- driving below approximately 32 km/h or while driving in reverse

Main Scenario
- Car emit sound out_27, because EV operate quietly, and this sound helps to alert pedestrians of your oncoming vehicle

Post conditions
- Pedestrians know that car is near
"""

# Prepare the input for the T5 model
input_ids = tokenizer.encode("summarize: " + input_text, return_tensors='pt', max_length=512, truncation=True)

# Generate the summary
summary_ids = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Summary:")
print(summary)


Summary:
Preconditions - driving below approximately 32 km/h or while driving in reverse Main Scenario - Car emit sound out_27, because EV operate quietly, and this sound helps to alert pedestrians of your oncoming vehicle.


In [21]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Initialize the T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Your input text
input_text = """
Acoustics characteristics
The sound emitted by the vehicle type submitted for approval shall be measured by the methods described in Annex 3 to this Regulation.
The specifications of this Regulation are applicable for the speed range of greater than 0 km/h up to and inclusive 20 km/h. Operation of an AVAS is permitted at vehicle speeds outside the specification range. AVAS may be operational independent of the operation of an internal combustion engine inside or outside of the specified operation range.
If the vehicle that is not equipped with an AVAS fulfils the overall levels as specified in Table 2 below with a margin of +3 dB(A), the specification for one-third octave bands and the frequency shift do not apply.an internal combustion engine inside or outside of the specified operation range. 
Frequency shift to signify acceleration and deceleration
The intention of frequency shift is to acoustically inform road users about the change in vehicle speed.
"""

# Prepare the input for the T5 model
input_ids = tokenizer.encode("summarize: " + input_text, return_tensors='pt', max_length=512, truncation=True)

# Generate the summary
summary_ids = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Summary:")
print(summary)


Summary:
the specifications of this Regulation are applicable for the speed range of greater than 0 km/h up to and inclusive 20 km/h. AVAS may be operational independent of the operation of an internal combustion engine inside or outside of the specified operation range.


In [10]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

input_text = """
Preconditions
- driving below approximately 32 km/h or while driving in reverse

Main Scenario
- Car emit sound out_27, because EV operate quietly, and this sound helps to alert pedestrians of your oncoming vehicle

Post conditions
- Pedestrians know that car is near
"""
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_ids = tokenizer(
    [WHITESPACE_HANDLER(input_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

ConnectTimeout: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Max retries exceeded with url: /csebuetnlp/mT5_multilingual_XLSum/ef78f86560d809067d12bac6c09f19a462cb3af3f54d2b8acbba26e1433125d6?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27spiece.model%3B+filename%3D%22spiece.model%22%3B&Expires=1729002663&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyOTAwMjY2M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9jc2VidWV0bmxwL21UNV9tdWx0aWxpbmd1YWxfWExTdW0vZWY3OGY4NjU2MGQ4MDkwNjdkMTJiYWM2YzA5ZjE5YTQ2MmNiM2FmM2Y1NGQyYjhhY2JiYTI2ZTE0MzMxMjVkNj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=kjchZElOJIeXsfq202ethbOIolwYXKcZK1QFCoC7FcagxYLtCqKP7FeFIKrFamoCFist68WLWTSL5s1qauAi8kaN3gTyscDI3BTlqArKhIXLPiyV9atEoFBJpHmH38QQylsMv72wbLHg0Y1ozPi-piVAAHgxORjLpZzHN8Y6P0tSL2waoMuzmWb~e~UrAavPcAtJ4eWJQpoUmgIND0O-0JdInq0OvuEEgXgKmkp4gtHe9R-mCemf2DN4qvjzUM3k9FzaMvGV0fxMoiyej4dMAjHQ~qmyzIzuYmTNBsH4a2UDMSLNb6-V265A-lbV